In [52]:
import bs4 as bs
import urllib.request
import pandas as pd

tweet_data = pd.read_csv("question_responce.csv")

In [58]:
tweet_data.responce[2]

' Aww thats definitely a future pilot in the making HappyHalloween'

In [57]:
tweet_data['question'] = tweet_data['question'].map(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", '',x))
tweet_data['responce'] = tweet_data['responce'].map(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", '',x))

In [60]:
import nltk
response_sentences = nltk.sent_tokenize(str(tweet_data.responce))

In [61]:
print(response_sentences)

['0        Well be sure to pass along your kind words AA...\n1        Our apologies for the delay in responding to ...\n2        Aww thats definitely a future pilot in the ma...\n3                         Were sorry for your frustration\n4        Were glad you got to kick back and enjoy a sh...\n                              ...                        \n1847     We know staying connected is important why no...\n1848     Weve capped our fares for nonstop flights at ...\n1849     Please give our Baggage team a call at 800866...\n1850     Our apologies for the hold Our Central Baggag...\n1851     Were providing waivers for St Croix Gillian P...\nName: responce, Length: 1852, dtype: object']


In [64]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def chatbot_answer(user_query):
    
    #Append the query to the sentences list
    response_sentences.append(user_query)
    vectorizer = TfidfVectorizer()
    sentences_vectors = vectorizer.fit_transform(response_sentences)
    
    #Measure the cosine similarity and take the second closest index because the first index is the user query
    vector_values = cosine_similarity(sentences_vectors[-1], sentences_vectors)
    answer = response_sentences[vector_values.argsort()[0][-2]]
    #Final check to make sure there are result present. If all the result are 0, means the text input by us are not captured in the corpus
    input_check = vector_values.flatten()
    input_check.sort()
    
    if input_check[-2] == 0:
        return "Please Try again"
    else: 
        return answer

In [65]:
print("Hello, I am the Airplane Chatbot. How can I help you ?")
while(True):
    query = input().lower()
    if query not in ['bye', 'good bye', 'take care']:
        print("Chatbot: ", end="")
        oui = chatbot_answer(query)
        response_sentences.remove(query)
    else:
        print("See You Again")
        break

with open("Output.txt", "w") as text_file:
    text_file.write(oui)


Hello, I am the Airplane Chatbot. How can I help you ?
Chatbot: answer 0        Well be sure to pass along your kind words AA...
1        Our apologies for the delay in responding to ...
2        Aww thats definitely a future pilot in the ma...
3                         Were sorry for your frustration
4        Were glad you got to kick back and enjoy a sh...
                              ...                        
1847     We know staying connected is important why no...
1848     Weve capped our fares for nonstop flights at ...
1849     Please give our Baggage team a call at 800866...
1850     Our apologies for the hold Our Central Baggag...
1851     Were providing waivers for St Croix Gillian P...
Name: responce, Length: 1852, dtype: object
input check [0. 1.]
query hello
responces sentences ['0        Well be sure to pass along your kind words AA...\n1        Our apologies for the delay in responding to ...\n2        Aww thats definitely a future pilot in the ma...\n3               